# Community detection in Graphs

We will explore the different community detection algorithms in graphs. 

Not all fo them fit with our problem requirements, we will focus on the algorithms which fit the following requirements
* Directed edges.
* Homogeneous nodes. 
* Possible Weighted relationships (in the future).

The algorithms which fit the requirements are:
* Conductance metric
* K-1 Coloring
* Label Propagation
* Louvain
* Modularity metric
* Modularity Optimization
* Strongly Connected Components
* Weakly Connected Components

We will use an additional approach after embedding the graph, we have use Node2Vector and DeepWalk
* K-Means
* DBSCAN

We will explore each one of them, how they work, their advantages and disadvantages and when to use them. 

Check this documentation to understand and explore the Community Detection algorithms:
https://neo4j.com/docs/graph-data-science/current/algorithms/community/

### Conductance metric 

### K-Means Clustering for Community Detection 
Read the embedding graph and apply clustering with cuML (RAPIDS) to use GPU

In [1]:
import torch

In [ ]:
# Read embedding representation
node2vector_embeddings_filepath = 'node2vector_embeddings.pt'
node2vector_embeddings = torch.load(node2vector_embeddings_filepath, weights_only=True)
node2vector_embeddings = node2vector_embeddings.cpu().numpy()

In [ ]:
# Init k-means clustering
from sklearn.cluster import KMeans

In [ ]:
n_clusters = 100  # N Clusters

kmeans_model = KMeans(
    n_clusters=n_clusters,
    max_iter=300,
    tol=1e-4,
    random_state=42
)


kmeans_model.fit(node2vector_embeddings)

labels = kmeans_model.labels_
print(labels)

In [ ]:
labels

In [ ]:
len(labels)

In [7]:
len(labels)

19129